In [2]:
# gmailer init


In [1]:
import datetime as dt

import yaml

import requests
import smtplib
import ssl
import mimetypes
from email.message import EmailMessage

import pandas as pd



## Load Configurations

In [8]:
## password and config
config = "../config.yaml"
config = yaml.safe_load(open(config))

def parse_config(config: dict):
    """
    Parse the configuration yaml.

    Parameters
    -------
    config (dict{any}): Dictionary of congiruations.

    Returns
    -------
    credentials (dict(any)): Dictionary of credentials.
    """
    credentials = config["credentials"]
    ...
    return credentials

credentials = parse_config(config)

# Methods

In [3]:
def get_csv_as_list(filepath: str) -> list:
    """
    Get list using a path to a csv.

    Parameters
    -------
    filepath (str): filepath. 

    Returns
    -------
    ([str]): List of values.
    
    """
    import pandas as pd
    recipients = pd.read_csv(recipients_csv_path, header=None)
    return recipients[0].values.tolist()

# Email class

In [5]:
sender = "jaime.meriz13@gmail.com"

# recipient path should be: list of recipients. Single recipient.
recipients_csv_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/email_recipients.csv"
# recipients_csv_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/email_recipients_2.csv"
recipients = get_csv_as_list(recipients_csv_path)

subject = "DEV" + "-" "Jmail" + "-" + str(dt.date.today())
body = "body\nyours\nDr. Lopez"
# password = "password"

In [48]:

def send_mail(sender: str, recipients: list, subject:str, body: str, password: str):
    """
    Send an email via SMTP.

    Parameters
    -------
    sender (str): Sender as a string.
    recipients ([str]): List of recipients. 
    subject (str): Email subject line.
    body (str): Email body as string.
    password (str): Gmail password as string.
    
    Returns
    -------
    None
    """

    email = EmailMessage()
    email["Sender"] = sender
    email["Recipients"] = ", ".join(recipients)
    email["Subject"] = subject
    email.set_content(body)


    context = ssl.create_default_context()
    
    # Email failsafe.
    confirm_send = input(f"Are you sure you want to send to recipients? (Y/N) \n\n {recipients}\n")

    if confirm_send.lower() == "y":
        print("Sending email...")
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp_server:
            smtp_server.login(sender, password)
            smtp_server.sendmail(sender, recipients, email.as_string())
        print("Message sent!")
    else:
        print("Message NOT sent!")


    return email

password = credentials["gmail"]["password"]
email = send_mail(sender, recipients, subject, body, password)


Sending email...
Message sent!


# Name lookup using email

In [64]:
# recipients

# # now look them up with clearbit.

# # get the api key
# clearbit_api_key = config["credentials"]["clearbit"]["api_key"]
# clearbit_api_key

# clearbit_api_key


# # form your request
# username = clearbit_api_key
# password = None
# auth = (username, password)


# email = recipients[0]
# url  =  f"https://person.clearbit.com/v2/combined/find?email=:{email}"
# print("url: ", url)


# # r = requests.get('https://api.github.com/user', auth=('user', 'pass'))

# # request
# x = requests.get(url, auth=auth)
# x = x.json()


# NOTE: This should be part of a class.
def get_name(response: requests.models.Response) -> (str, str):
    """
    Parse response for HTTP request.

    Parameters
    -------
    response (requests.models.Response): Http response with person data.

    Returns
    -------
    name (str, str): Tuple of first and last name. 
    """
    if not isinstance(y, requests.models.Response):
        print("Not a request.")
        return 
    name = (None, None)
    data = response.json()
    person = data["person"]
    first_name = person["name"]["givenName"]
    last_name = person["name"]["familyName"]
    name = (first_name, last_name)
    return name


get_name(y)


('Marco', 'Starger')

In [72]:
def get_response(url: str, username=None, password=None, api_key=None):
    """
    Get a response from API using HTTP.

    Parameters
    -------
    url (str): Url for API request.
    username (str): Optional. Username.
    password (str): Optional. Password.
    api_key (str): API key.

    Returns
    -------
    api_response (requests.models.Response): Response.
    """
    api_response = None
    api_response = requests.get(url, auth=(username, password))
    return api_response

    
clearbit_api_key = config["credentials"]["clearbit"]["api_key"]
email = recipients[0]
url = f"https://person.clearbit.com/v2/combined/find?email=:{email}"

# Note: read docs on clearbit. Makes this look really funky.
clearbit_response = get_response(url, username=clearbit_api_key, password=None, api_key=None)
get_name(clearbit_response)


('Marco', 'Starger')

In [70]:
type(clearbit_response)

requests.models.Response